In [18]:
# Copyright UCL Business plc 2017. Patent Pending. All rights reserved.
#
# The MonoDepth Software is licensed under the terms of the UCLB ACP-A licence
# which allows for non-commercial use only, the full terms of which are made
# available in the LICENSE file.
#
# 
# Modified by Yongyang Nie.
# Copyright (c) by Yongyang Nie 
# All Rights Reserved
# Contact: contact@neilnie.com
#

from __future__ import absolute_import, division, print_function

# only keep warnings and errors
import os
import sys
import argparse
import time
import pandas as pd
import cv2
import tensorflow as tf
import tensorflow.contrib.slim as slim
import scipy.misc
import matplotlib.pyplot as plt

from monodepth_model import *
from monodepth_dataloader import *
from average_gradients import *


## Helper Methods

In [14]:
def post_process_disparity(disp):
    _, h, w = disp.shape
    l_disp = disp[0,:,:]
    r_disp = np.fliplr(disp[1,:,:])
    m_disp = 0.5 * (l_disp + r_disp)
    l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
    l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
    r_mask = np.fliplr(l_mask)

    return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp

# github.com/aubricus
def print_progress(iteration, total, prefix='', suffix='', decimals=1, bar_length=100):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        bar_length  - Optional  : character length of bar (Int)
    """
    str_format = "{0:." + str(decimals) + "f}"
    percents = str_format.format(100 * (iteration / float(total)))
    filled_length = int(round(bar_length * iteration / float(total)))
    bar = '█' * filled_length + '-' * (bar_length - filled_length)

    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percents, '%', suffix)),

    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()

## Define Parameters

In [2]:
checkpoint_path = '/home/neil/Workspace/monodepth/models/cityscape/model_cityscapes.data-00000-of-00001'
input_height = 256
input_width =512 

## Setting Up TensorFlow

In [3]:
params = monodepth_parameters(
        encoder='vgg',
        height=input_height,
        width=input_width,
        batch_size=2,
        num_threads=1,
        num_epochs=1,
        do_stereo=False,
        wrap_mode="border",
        use_deconv=False,
        alpha_image_loss=0,
        disp_gradient_loss_weight=0,
        lr_loss_weight=0,
        full_summary=False)

left  = tf.placeholder(tf.float32, [2, input_height, input_width, 3])
model = MonodepthModel(params, "test", left, None)

# SESSION
config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)

# SAVER
train_saver = tf.train.Saver()

# INIT
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

# RESTORE
restore_path = checkpoint_path.split(".")[0]
train_saver.restore(sess, restore_path)

INFO:tensorflow:Restoring parameters from /home/neil/Workspace/monodepth/models/cityscape/model_cityscapes


## Create a new directory for depth images

In [5]:
dirs = os.listdir('/hdd/ssd_2/dataset/segmentation/training')
print("making new directories... \n" + str(dirs))

for d in dirs:
    if (os.path.isdir('/hdd/ssd_2/dataset/segmentation/training_depth/' + d)):
        print("directory: " + d + " already exists")
    else:
        os.mkdir('/hdd/ssd_2/dataset/segmentation/training_depth/' + d)

making new directories... 
['krefeld', 'zurich', 'monchengladbach', 'freiburg', 'augsburg', 'duisburg', 'muhlheim-ruhr', 'schweinfurt', 'bad-honnef', 'nuremberg', 'darmstadt', 'ulm', 'konstanz', 'hamburg', 'saarbrucken', 'wuppertal', 'wurzburg', 'bayreuth', 'bochum', 'erfurt', 'bremen', 'stuttgart', 'dusseldorf', 'mannheim', 'jena', 'konigswinter', 'erlangen', 'troisdorf', 'strasbourg', 'heilbronn', 'hanover', 'cologne', 'dortmund', 'heidelberg', 'oberhausen', 'bamberg', 'karlsruhe', 'dresden', 'tubingen', 'weimar', 'aachen']
directory: krefeld already exists
directory: zurich already exists
directory: monchengladbach already exists
directory: freiburg already exists
directory: augsburg already exists
directory: duisburg already exists
directory: muhlheim-ruhr already exists
directory: schweinfurt already exists
directory: bad-honnef already exists
directory: nuremberg already exists
directory: darmstadt already exists
directory: ulm already exists
directory: konstanz already exists
di

## Loop through all images

In [ ]:
labels = pd.read_csv('/hdd/ssd_2/dataset/segmentation/labels.csv').values
i = 0
j = 0

for row in labels:
    
    if i > 6416:
        image_path = row[0]
    
        input_image = scipy.misc.imread(image_path, mode="RGB")
        original_height, original_width, num_channels = input_image.shape
        input_image = scipy.misc.imresize(input_image, [input_height, input_width], interp='lanczos')
        input_image = input_image.astype(np.float32) / 255
        input_images = np.stack((input_image, np.fliplr(input_image)), 0)

        disp = sess.run(model.disp_left_est[0], feed_dict={left: input_images})
        disp_pp = post_process_disparity(disp.squeeze()).astype(np.float32)
    
        new_image_path = image_path.replace("training", "training_depth")

        disp_to_img = scipy.misc.imresize(disp_pp.squeeze(), [original_height, original_width])
        plt.imsave(new_image_path, disp_to_img, cmap='gray')
        j = j + 1
        print_progress(j, 16557, prefix='Progress:', suffix='Complete', bar_length = 50)
    
    i = i + 1
        
print("data finished processing")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Progress: |█-------------------------------------------------| 2.5% Complete